# 四、分布式数据并行训练

### 一 分布式数据并行训练（Distributed Data Parallel Training）

在作业的这一部分中，我们将探索使用多张 GPU 训练语言模型的方法，重点关注**数据并行（data parallelism）**。
我们将首先对 PyTorch 中的分布式通信进行一个入门介绍。接着，我们会学习一种朴素（naive）的分布式数据并行训练实现方式，并在此基础上实现和评测多种用于提升通信效率的改进方案。

---

### 1.1 PyTorch 中的单机分布式通信（Single-Node Distributed Communication in PyTorch）

我们先来看一个 PyTorch 中的简单分布式应用，其目标是：
**生成四个随机整数张量，并计算它们的和。**

在下面的分布式示例中，我们会启动 **4 个 worker 进程**，每个进程都会生成一个随机整数张量。
为了对这些进程中的张量求和，我们将调用 **all-reduce** 这一集合通信（collective communication）操作。

`all-reduce` 的作用是：

> 将所有进程中的张量进行规约（这里是求和），并用规约后的结果 **原地替换** 每个进程中的原始张量。

也就是说，在 all-reduce 完成后，每个进程中的张量都会变成相同的“全局求和结果”。

下面我们来看代码。

---

在运行下面的脚本后，我们会得到如下输出。
可以看到：在 all-reduce 之前，每个 worker 进程持有的张量都不同；而在 all-reduce 操作之后，由于所有进程中的张量被求和并同步，每个进程中的 `data` 都被**原地修改**为相同的结果张量。

---

如果你多次运行这个脚本，你会发现打印输出的顺序并不是确定的。这是因为该应用运行在分布式环境中，我们无法控制各个进程执行打印语句的精确顺序。
**唯一可以保证的是**：在 all-reduce 操作完成之后，所有进程都会持有**逐位（bitwise）完全一致**的结果张量。

---




In [1]:
%%writefile distributed_demo.py
# =========================
# 标准库 & PyTorch 相关导入
# =========================

import os
import torch
import torch.distributed as dist
import torch.multiprocessing as mp


# =========================
# 分布式环境初始化函数
# =========================
def setup(rank, world_size):
    """
    初始化分布式通信环境（Process Group）

    参数：
    - rank: 当前进程的编号（0 ~ world_size-1）
    - world_size: 总进程数
    """

    # 指定“主进程”的地址
    # 所有进程都会通过这个地址进行 rendezvous（集合）
    # 单机多进程时使用 localhost
    os.environ["MASTER_ADDR"] = "localhost"

    # 指定主进程监听的端口号
    # 所有进程必须保持一致
    os.environ["MASTER_PORT"] = "29500"

    # 初始化进程组（非常关键的一步）
    # backend="gloo"：使用 gloo 通信后端（CPU 通用，支持多平台）
    # rank：当前进程在所有进程中的唯一编号
    # world_size：参与通信的总进程数
    dist.init_process_group(
        backend="gloo",
        rank=rank,
        world_size=world_size
    )


# =========================
# 每个进程实际执行的函数
# =========================
def distributed_demo(rank, world_size):
    """
    每个进程都会执行这个函数一次
    """

    # 初始化分布式通信环境
    setup(rank, world_size)

    # 每个进程各自生成一个本地张量
    # torch.randint 是“本地操作”，不同 rank 得到的值不同
    # 这里生成一个 shape 为 (3,) 的一维张量
    data = torch.randint(0, 10, (3,))

    # 打印 all-reduce 之前的数据
    # 注意：由于是多进程并行执行，打印顺序不确定
    print(f"rank {rank} data (before all-reduce): {data}")

    # 核心操作：all-reduce
    # - 对所有 rank 上的 data 做规约（默认是 SUM）
    # - 结果会“原地”写回到每个 rank 的 data 中
    # - async_op=False 表示同步执行（阻塞，直到完成）
    dist.all_reduce(data, async_op=False)

    # 打印 all-reduce 之后的数据
    # 此时每个 rank 的 data 都是完全相同的
    print(f"rank {rank} data (after all-reduce): {data}")


# =========================
# 程序主入口
# =========================
if __name__ == "__main__":

    # world_size 表示总进程数
    # 在 DDP 中通常等于 GPU 数量
    world_size = 4

    # 使用 torch.multiprocessing.spawn 启动多进程
    mp.spawn(
        fn=distributed_demo,   # 每个进程要执行的函数
        args=(world_size,),    # 传给函数的额外参数（rank 会自动作为第一个参数）
        nprocs=world_size,     # 启动的进程数量
        join=True              # 主进程是否等待所有子进程结束
    )


Overwriting distributed_demo.py


由于分布式环境不支持ipynb的notebook格式，只能先写入.py文件再在notebook中运行

In [2]:
!python distributed_demo.py


[Gloo] Rank [Gloo] Rank [Gloo] Rank [Gloo] Rank 021 is connected to 3 is connected to  is connected to 3 is connected to 33 peer ranks. 3 peer ranks.  peer ranks. Expected number of connected peer ranks is :  peer ranks. Expected number of connected peer ranks is : Expected number of connected peer ranks is : 3Expected number of connected peer ranks is : 33

3

rank 2 data (before all-reduce): tensor([1, 8, 8])rank 3 data (before all-reduce): tensor([2, 6, 8])rank 0 data (before all-reduce): tensor([0, 3, 4])


rank 1 data (before all-reduce): tensor([1, 9, 3])
rank 0 data (after all-reduce): tensor([ 4, 26, 23])
rank 3 data (after all-reduce): tensor([ 4, 26, 23])
rank 2 data (after all-reduce): tensor([ 4, 26, 23])rank 1 data (after all-reduce): tensor([ 4, 26, 23])



## 这段代码在“分布式视角”下做了什么？

> **它在一台机器上启动 4 个独立进程，每个进程生成一个随机张量，然后通过 `all_reduce` 把所有进程的张量求和，并让每个进程都拿到相同的结果。**

这正是 **数据并行训练中“梯度同步”**的最小原型。

---

## 二、整体执行流程（非常重要）

当运行：

```bash
python distributed_hello_world.py
```

实际发生的是： **主进程启动**、 `mp.spawn` 启动 **4 个子进程**。

每个子进程 被分配一个唯一的 `rank`（0～3），加入同一个 `process group`， 各自生成数据， 通过 `all_reduce` 进行通信。

注意：**每个 rank 是一个独立的 Python 进程，不是线程**

---

## 三、逐段精讲代码

---

### 1 导入模块

```python
import os
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
```

作用拆解：

| 模块                      | 作用             |
| ----------------------- | -------------- |
| `os`                    | 设置分布式通信所需的环境变量 |
| `torch`                 | 张量与随机数         |
| `torch.distributed`     | 分布式通信核心 API    |
| `torch.multiprocessing` | 启动多个进程         |

---

### 2 setup：初始化分布式通信

```python
def setup(rank, world_size):
    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "29500"
    dist.init_process_group("gloo", rank=rank, world_size=world_size)
```

这是 **分布式程序的“入群”步骤**。

#### （1）`MASTER_ADDR` & `MASTER_PORT`

```text
谁是“总协调者”（Rendezvous Server）
```

* 所有进程都会连到这个地址
* 单机多进程 → 用 `localhost`
* 多机 → 用主节点 IP

---

#### （2）`init_process_group`

```python
dist.init_process_group(
    backend="gloo",
    rank=rank,
    world_size=world_size
)
```

这是最关键的一行。

参数解释：

| 参数               | 含义             |
| ---------------- | -------------- |
| `backend="gloo"` | 通信后端（CPU / 通用） |
| `rank`           | 当前进程的唯一编号      |
| `world_size`     | 进程总数           |

 **核心概念**：

* `rank`：我是谁
* `world_size`：我们一共有多少人

在 DDP 里：

* **1 个 GPU ≈ 1 个 rank**

---

### 3  distributed_demo：每个进程执行的逻辑

```python
def distributed_demo(rank, world_size):
    setup(rank, world_size)
```

> 每个子进程都会执行这个函数一次

---

#### 3.1 生成本地数据

```python
data = torch.randint(0, 10, (3,))
```

* 每个 rank 生成 **不同的随机张量**
* 例如：

  * rank 0 → `[4, 4, 7]`
  * rank 1 → `[9, 5, 3]`
  * rank 2 → `[6, 0, 7]`
  * rank 3 → `[3, 7, 8]`

---

#### 3.2 all-reduce 前打印

```python
print(f"rank {rank} data (before all-reduce): {data}")
```

 打印顺序**不保证**，因为 4 个进程是并行的。

---

#### 3.3 核心：`all_reduce`

```python
dist.all_reduce(data, async_op=False)
```

这是整段代码的**灵魂**。

### all-reduce 做了什么？

对所有 rank 上的 `data`：

1. **收集**
2. **求和（默认是 SUM）**
3. **把结果写回每个 rank 的 `data`（原地）**

数学上等价于：

```python
data = data_rank0 + data_rank1 + data_rank2 + data_rank3
```

关键点：

* **in-place 操作**
* 所有 rank 最终结果 **完全一致**
* 默认是 `SUM`（梯度同步就是用这个）

---

#### 3.4 all-reduce 后打印

```python
print(f"rank {rank} data (after all-reduce): {data}")
```

你会看到所有 rank 都变成：

```text
tensor([22, 16, 25])
```

---

### 4 主入口：spawn 多进程

```python
if __name__ == "__main__":
    world_size = 4
```

* 定义进程数
* 在 DDP 中通常 = GPU 数量

---

```python
mp.spawn(
    fn=distributed_demo,
    args=(world_size,),
    nprocs=world_size,
    join=True
)
```

解释：

| 参数          | 含义           |
| ----------- | ------------ |
| `fn`        | 每个进程执行的函数    |
| `args`      | 传给函数的参数      |
| `nprocs`    | 启动多少个进程      |
| `join=True` | 主进程等待所有子进程结束 |

`spawn` 会自动给每个进程分配 `rank = 0, 1, 2, 3`，把 `rank` 作为 **第一个参数** 传入 `fn`。

---

## 四、这段代码 ↔ 真正的 DDP 训练

| 本例                 | DDP 中            |
| ------------------ | ---------------- |
| `data`             | 梯度（`param.grad`） |
| `torch.randint`    | 前向 + 反向计算        |
| `all_reduce(data)` | 梯度同步             |
| 单次操作               | 每个 iteration 都做  |

本质上：

> **DDP = 前向各算各的，反向之后用 all-reduce 同步梯度**




## 2.1.1 分布式应用基准测试的最佳实践

（Best Practices for Benchmarking Distributed Applications）

在本作业的这一部分中，你将对**分布式应用进行基准测试**，以更好地理解**通信带来的开销**。以下是一些推荐的最佳实践：

* **尽可能在同一台机器上运行基准测试**，以便进行可控、公平的对比。

* **在正式计时之前进行多次 warm-up（预热）**。
  这一点对 **NCCL 通信调用**尤为重要。通常 **5 次预热迭代** 就足够了。

* **在 GPU 上进行基准测试时，务必调用 `torch.cuda.synchronize()`**，以等待 CUDA 操作真正完成。
  注意：即使在通信操作中设置了 `async_op=False`，这一步仍然是必须的，因为该调用只保证操作被**加入 GPU 队列**，而不是通信实际完成。³

* **不同 rank 的测量时间可能略有差异**，因此通常会在所有 rank 之间**聚合测量结果**以提高估计的稳定性。
  可以使用 **all-gather 集合通信操作**（特别是 `dist.all_gather_object`）来收集所有 rank 的结果。

* **一般建议先在 CPU 上使用 Gloo 后端进行本地调试**，然后在实际问题中根据需要使用 GPU + NCCL 进行基准测试。
  在两种后端之间切换通常只需要修改：

  * `init_process_group` 中的 backend 参数
  * 张量的设备（CPU / GPU）转换

 


# 作业一

## 问题（distributed_communication_single_node）：5 分

请编写一个脚本，用于在**单节点多进程（single-node multi-process）**设置下，**基准测试 all-reduce 操作的运行时间**。

你可以参考前面给出的示例代码作为起点，并尝试改变以下配置参数：

### 实验设置要求

* **后端 + 设备类型**

  * Gloo + CPU
  * NCCL + GPU

* **all-reduce 数据规模**

  * 数据类型：`float32`
  * 张量大小：

    * 1MB
    * 10MB
    * 100MB
    * 1GB

* **进程数量**

  * 2 个进程
  * 4 个进程
  * 6 个进程

### 资源限制

* 最多可使用 **6 张 GPU**
* 每一次基准测试运行时间 **不得超过 5 分钟**

---

## 交付内容（Deliverable）

* 提供 **图表（plot）和/或表格（table）**，用于比较不同实验设置下的性能表现
* 并附上 **2–3 句话的分析说明**，总结你的实验结果，以及你对各因素（后端、数据规模、进程数等）如何相互影响的理解


In [3]:

%%writefile distributed_communication_single_node_demo.py
import os
import time
import torch
import torch.distributed as dist
import argparse
import torch
from torch.multiprocessing import spawn


# ============================================================
# 1. 分布式环境初始化与清理
# ============================================================

def init_distributed_environment(
    master_addr: str,
    master_port: int,
    global_rank: int,
    world_size: int,
    backend: str
):
    """
    初始化 PyTorch 分布式通信环境（进程组）

    参数说明：
    - master_addr : rank 0 所在节点的 IP 地址
    - master_port : 用于进程间通信的端口
    - global_rank : 当前进程在所有进程中的唯一编号
    - world_size  : 总进程数
    - backend     : 通信后端（gloo / nccl / mpi）
    """

    # --------------------------------------------------------
    # 1.1 设置 rendezvous（所有进程汇合的地址）
    # --------------------------------------------------------
    # 所有进程必须通过 MASTER_ADDR:MASTER_PORT 建立初始连接
    os.environ["MASTER_ADDR"] = master_addr
    os.environ["MASTER_PORT"] = str(master_port)

    # --------------------------------------------------------
    # 1.2 初始化进程组（分布式的“总开关”）
    # --------------------------------------------------------
    # 这是使用 torch.distributed 的前置条件
    dist.init_process_group(
        backend=backend,
        rank=global_rank,
        world_size=world_size
    )


def destroy_distributed_environment():
    """
    清理分布式环境并释放资源
    """

    # 销毁进程组，防止资源泄漏或死锁
    dist.destroy_process_group()

    # GPU 场景下，清空 PyTorch 的 CUDA cache（非强制，但推荐）
    if torch.cuda.is_available():
        torch.cuda.empty_cache()


# ============================================================
# 2. All-Reduce 通信性能 Benchmark
# ============================================================

def run_all_reduce_benchmark(
    global_rank: int,
    world_size: int,
    tensor_size_mb: int,
    backend: str,
    device: str,
    master_addr: str,
    master_port: int
):
    """
    对 dist.all_reduce 进行性能测试

    测试指标：
    - 单次 all-reduce 平均耗时
    - 理论通信带宽（GB/s）
    """

    # --------------------------------------------------------
    # 2.1 初始化分布式环境
    # --------------------------------------------------------
    init_distributed_environment(
        master_addr=master_addr,
        master_port=master_port,
        global_rank=global_rank,
        world_size=world_size,
        backend=backend
    )

    # --------------------------------------------------------
    # 2.2 绑定 GPU（仅在 CUDA 场景下）
    # --------------------------------------------------------
    if device == "cuda":
        # 通常约定：rank i → GPU i
        torch.cuda.set_device(global_rank)
        torch.cuda.empty_cache()

    # --------------------------------------------------------
    # 2.3 构造通信测试用的 Tensor
    # --------------------------------------------------------
    # 将 MB 转换为字节
    tensor_num_bytes = tensor_size_mb * 1024 * 1024

    # float32 占 4 字节
    bytes_per_element = 4
    num_elements = tensor_num_bytes // bytes_per_element

    # 随机生成测试数据（数值本身不重要）
    communication_tensor = torch.randn(
        num_elements,
        device=device,
        dtype=torch.float32
    )

    # --------------------------------------------------------
    # 2.4 Warm-up（非常重要）
    # --------------------------------------------------------
    # 原因：
    # - NCCL 通信器初始化
    # - CUDA kernel lazy initialization
    # - GPU 频率爬升
    warmup_iterations = 5
    for _ in range(warmup_iterations):
        dist.all_reduce(
            communication_tensor,
            op=dist.ReduceOp.SUM
        )

        # CUDA 是异步执行，必须显式同步
        if device == "cuda":
            torch.cuda.synchronize()

    # 确保所有进程在同一时刻开始正式测试
    dist.barrier()

    # --------------------------------------------------------
    # 2.5 正式 benchmark（计时）
    # --------------------------------------------------------
    benchmark_iterations = 20

    start_time = time.time()

    for _ in range(benchmark_iterations):
        dist.all_reduce(
            communication_tensor,
            op=dist.ReduceOp.SUM
        )

    # GPU 场景下等待所有 kernel 完成
    if device == "cuda":
        torch.cuda.synchronize()

    end_time = time.time()

    # --------------------------------------------------------
    # 2.6 性能指标计算
    # --------------------------------------------------------
    total_elapsed_time = end_time - start_time
    avg_latency_seconds = total_elapsed_time / benchmark_iterations

    # 带宽 = 数据量 / 时间（单位：GB/s）
    bandwidth_gbps = (tensor_num_bytes / avg_latency_seconds) / 1e9

    # --------------------------------------------------------
    # 2.7 打印结果（仅 rank 0）
    # --------------------------------------------------------
    if global_rank == 0:
        print(
            f"[All-Reduce Benchmark]\n"
            f"  Backend        : {backend}\n"
            f"  Device         : {device}\n"
            f"  World Size     : {world_size}\n"
            f"  Tensor Size    : {tensor_size_mb} MB\n"
            f"  Avg Latency    : {avg_latency_seconds * 1000:.4f} ms\n"
            f"  Bandwidth      : {bandwidth_gbps:.4f} GB/s\n"
        )

    # --------------------------------------------------------
    # 2.8 汇总所有 rank 的结果（教学用）
    # --------------------------------------------------------
    local_result = {
        "rank": global_rank,
        "world_size": world_size,
        "backend": backend,
        "device": device,
        "tensor_size_mb": tensor_size_mb,
        "avg_latency_ms": avg_latency_seconds * 1000,
        "bandwidth_gbps": bandwidth_gbps,
    }

    gathered_results = [None for _ in range(world_size)]
    dist.all_gather_object(gathered_results, local_result)

    # --------------------------------------------------------
    # 2.9 清理环境
    # --------------------------------------------------------
    destroy_distributed_environment()

    # 只让主进程返回结果
    if global_rank == 0:
        return gathered_results


def main():

    for size in [1,10,100,1000]:
        for Backend in ["gloo", "nccl"]:
            for world_size in [1]:# 有多少个GPU就增加测试用例，当然一个也能跑
                parser = argparse.ArgumentParser("All-Reduce Benchmark")

                parser.add_argument("--world_size", type=int, default=world_size,
                                    help="进程总数（通常等于 GPU 数）")
                parser.add_argument("--tensor_size_mb", type=int, default=size,
                                    help="通信 tensor 大小（MB）")
                parser.add_argument("--backend", type=str, default=Backend,
                                    choices=["gloo", "nccl"],
                                    help="分布式通信后端")
                parser.add_argument("--device", type=str, default="cuda",
                                    choices=["cpu", "cuda"],
                                    help="运行设备")
                parser.add_argument("--master_addr", type=str, default="127.0.0.1")
                parser.add_argument("--master_port", type=int, default=29500)

                args = parser.parse_args()

                # GPU 数量检查
                if args.device == "cuda":
                    assert torch.cuda.is_available(), "CUDA 不可用"
                    assert args.world_size <= torch.cuda.device_count(), (
                        "world_size 不能超过 GPU 数量"
                    )

                # 使用 spawn 启动多进程
                spawn(
                    fn=run_all_reduce_benchmark,
                    args=(
                        args.world_size,
                        args.tensor_size_mb,
                        args.backend,
                        args.device,
                        args.master_addr,
                        args.master_port
                    ),
                    nprocs=args.world_size,
                    join=True
                )


if __name__ == "__main__":
    main()



Overwriting distributed_communication_single_node_demo.py


In [4]:
!python distributed_communication_single_node_demo.py

[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[All-Reduce Benchmark]
  Backend        : gloo
  Device         : cuda
  World Size     : 1
  Tensor Size    : 1 MB
  Avg Latency    : 0.2600 ms
  Bandwidth      : 4.0336 GB/s

/mnt/d/code/项目/cs336/CS336-Chinese-co-construction/coursework/Assignment2_System/.venv/lib/python3.12/site-packages/torch/distributed/distributed_c10d.py:4876: UserWarning: barrier(): using the device under current context. You can specify `device_id` in `init_process_group` to mute this warning.
  warnings.warn(  # warn only once
[All-Reduce Benchmark]
  Backend        : nccl
  Device         : cuda
  World Size     : 1
  Tensor Size    : 1 MB
  Avg Latency    : 0.0162 ms
  Bandwidth      : 64.5348 GB/s

[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[All-Reduce Benchmark]
  Backend        : gloo
  Device         : cuda
  World Size     : 1
  Tensor Size    : 10 MB
  Avg Latency

## 一、Gloo和NCCL的基本概念

在 PyTorch 中，多进程/多 GPU 之间进行通信（比如 `all_reduce`, `broadcast`, `all_gather`）需要 **通信后端（backend）**。
常用的两种是：

| 名称       | 适用设备            | 特点                                                 |
| -------- | --------------- | -------------------------------------------------- |
| **Gloo** | CPU / GPU（有限支持） | 跨平台，支持 CPU，多节点可以用 TCP/IP 通信，适合 CPU 张量，简单可靠         |
| **NCCL** | GPU（NVIDIA GPU） | NVIDIA 官方库，专门优化 GPU 间通信，支持高速 PCIe、NVLink、网络通信，性能最好 |

---

## 二、Gloo

Gloo 在 CPU 和 GPU 都可以用，但 GPU 支持有限，通常用在 CPU 张量上，它支持跨节点网络通信（TCP/IP）。并且使用方便，启动和调试比 NCCL 容易。在 CPU 环境下是**默认且可靠的选择**。

它通常使用在单机 CPU 多进程训练，或者小规模实验和调试，没有 GPU 的服务器环境。

在CPU 通信效率中等，GPU 上比 NCCL 慢，不适合大规模深度学习训练。

---

## 三、NCCL（NVIDIA Collective Communication Library）


它是专门为 NVIDIA GPU 设计的，支持多 GPU、多节点高速通信。并且优化了**PCIe 直连**，**NVLink（GPU 内部高速总线）**，**网络互联（Ethernet / InfiniBand）**，PyTorch 会自动使用 NCCL 的 ring / tree 通信算法做 `all_reduce`、`all_gather` 等操作。

通常在单机多 GPU 训练（如 2~8 GPU）上使用。分布式训练深度学习大模型，高性能 GPU 集群使用较多。

能将GPU 间带宽最大化，并且延迟极低（特别是大张量）， NCCL + CUDA 几乎是训练大模型的默认选择。

---

## 四、总结对比

| 特性           | Gloo            | NCCL               |
| ------------ | --------------- | ------------------ |
| 设备           | CPU（主要），GPU（有限） | GPU（NVIDIA 专用）     |
| 性能           | 中等              | 很高                 |
| 跨节点支持        | TCP/IP          | NVLink / PCIe / 网络 |
| 用途           | CPU 分布式 / 小实验   | GPU 多卡训练 / 大模型     |
| PyTorch 默认推荐 | CPU → Gloo      | GPU → NCCL         |




## 一、代码概览

这段代码的目的只有一个：

**在分布式环境下 benchmark `all_reduce` 的通信性能**
测量指标包括：单次 `all_reduce` 的平均耗时，实际通信带宽（GB/s）适用场景在多进程（多卡 / 多节点），对比不同 backend（`gloo` / `nccl`）， 对比 CPU vs GPU 通信性能。

## 二、模块介绍

### 1. `torch.distributed as dist`

它是PyTorch 的**分布式通信核心模块**，它提供进程组管理、collective ops（`all_reduce`, `broadcast`, `all_gather` 等）。

### 2. `torch.multiprocessing.spawn`

它是**多进程启动工具**，每个进程对应一个 `rank`，通常 rank + GPU id。当然分布式训练不等于多线程，**每个 rank 是一个独立 Python 进程**。


## 三、初始化分布式环境

```python
def setup(master_addr, master_port, rank, world_size, backend):
```

### 1. MASTER_ADDR / MASTER_PORT

```python
os.environ['MASTER_ADDR'] = master_addr
os.environ['MASTER_PORT'] = str(master_port)
```

定义**主节点（rank 0）的地址**，所有进程都通过这个地址进行 rendezvous，即使是**单机多卡**，也必须设置。

### 2. 初始化进程组

```python
dist.init_process_group(
    backend,
    rank=rank,
    world_size=world_size
)
```

| 参数         | 含义        |
| ---------- | --------- |
| backend    | 通信后端      |
| rank       | 当前进程的全局编号 |
| world_size | 总进程数      |

#### 常见 backend 对比

| backend | 适用场景            |
| ------- | --------------- |
| `gloo`  | CPU / 小规模       |
| `nccl`  | GPU / 高性能（事实标准） |
| `mpi`   | HPC 环境          |

---

## 四、`cleanup()`：资源回收

```python
def cleanup():
    dist.destroy_process_group()
```

它显式释放通信资源，防止程序异常退出导致死锁

```python
if torch.cuda.is_available():
    torch.cuda.empty_cache()
```

清空 CUDA cache（**不是释放显存，只是释放 PyTorch cache**），benchmark 中避免显存碎片影响。

---

## 五、`benchmark_all_reduce()`


### Step 1. 设置环境 & GPU 绑定

```python
setup(...)
```

#### GPU 场景下：

```python
torch.cuda.set_device(rank)
```

**为什么是 `rank`？**

通常约定：

```
rank 0 → GPU 0
rank 1 → GPU 1
...
```


### Step 2. 构造测试 Tensor

```python
tensor_size_bytes = tensor_size_mb * 1024 * 1024
num_elements = tensor_size_bytes // 4
tensor_data = torch.randn(num_elements, device=device)
```

`float32 = 4 bytes`，`randn` 只是为了填充数据，**值本身无关紧要**


### Step 3 Warm-up

```python
for _ in range(5):
    dist.all_reduce(tensor_data, op=dist.ReduceOp.SUM)
```

#### 为什么一定要 warm-up？

1. CUDA kernel lazy initialization
2. NCCL communicator 建立
3. GPU 时钟频率爬升
4. cache / pipeline 填充



GPU 场景下：

```python
torch.cuda.synchronize()
```

CUDA 是**异步执行**不同步，测到的是 launch 时间。

---

### Barrier：所有进程对齐

```python
dist.barrier()
```

确保所有 rank **同时开始计时**，防止快的进程提前进入 benchmark。


### Step 4. 正式 benchmark

```python
start_time = time.time()
```

```python
for _ in range(num_iterations):
    dist.all_reduce(tensor_data, op=dist.ReduceOp.SUM)
```

`num_iterations = 20`，用来取平均，降低噪声。

GPU 同步：

```python
torch.cuda.synchronize()
```

---

### Step 5. 计算耗时与带宽

#### 平均耗时

```python
avg_time = duration / num_iterations
```

#### 带宽计算

```python
bandwidth_gbps = (tensor_size_bytes / avg_time) / 1e9
```

含义：

```
GB/s = 数据量 / 时间
```

当然这是 **简化模型**，没区分 ring / tree，适合对比，不是绝对值。


### Step 6. 只让 rank 0 打印

```python
if rank == 0:
    print(...)
```

避免：N 个进程打印 N 份日志

---

### Step 7. `all_gather_object` 汇总结果

```python
dist.all_gather_object(gathered_results, local_result)
```

它支持 **Python 对象**，比 `all_gather(tensor)` 更灵活，常用于 benchmark / profiling，这样 rank 0 能拿到所有 rank 的结果。

---

### Step 8. 清理 & 返回结果

```python
cleanup()
```

```python
if rank == 0:
    return gathered_results
```

只有主进程返回， 子进程自动退出。




# 作业二

## 问题（naive_ddp）：5 分

**交付物**：编写一个脚本来通过在反向传播后对各个参数梯度进行全规约，以简单方式执行分布式数据并行训练。为了验证你的DDP实现的正确性，使用它在随机生成的数据上训练一个小玩具模型，并验证其权重与单进程训练的结果相匹配。

---

## 问题（naive_ddp_benchmarking）：3 分

在这种简单的DDP实现中，参数在每个反向传播后在各个rank之间单独进行全规约。为了更好地理解数据并行训练的开销，创建一个脚本来对之前实现的语言模型进行基准测试，该模型使用这种简单的DDP实现进行训练。测量每个训练步骤的总时间以及用于通信梯度的时间比例。收集在单节点设置（1个节点 x 2个GPU）下对XL模型的测量结果，如§1.1.2中所述。

**交付物**：描述你的基准测试设置，以及每个训练迭代的测量时间以及用于通信梯度的时间。

---

In [1]:
%%writefile ddp_model_demo.py

import os
import time
import random
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributed as dist

from torchvision import datasets, transforms
from torch.multiprocessing.spawn import spawn


# ============================
# 一、定义一个简单的全连接神经网络
# ============================
class SimpleNet(nn.Module):
    """
    一个最简单的三层全连接网络，用于 MNIST 分类
    输入：28×28 展平后的 784 维向量
    输出：10 类（0~9）
    """

    def __init__(self, input_dim: int, hidden_dim: int, num_classes: int):
        super().__init__()

        # 第一层全连接：输入层 -> 隐藏层
        self.fc1 = nn.Linear(input_dim, hidden_dim)

        # 第二层全连接：隐藏层 -> 隐藏层
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)

        # 输出层：隐藏层 -> 类别数
        self.fc3 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        """
        前向传播逻辑
        """
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        # 使用 log_softmax，方便后续配合 NLLLoss
        return F.log_softmax(x, dim=1)


# ============================
# 二、分布式环境初始化与清理
# ============================
def init_distributed_env(rank: int, world_size: int, backend: str):
    """
    初始化分布式进程组（所有进程都必须调用）
    """
    # 主进程地址和端口
    # 在单机多卡场景中，localhost 即可
    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "29500"

    # 初始化进程组
    dist.init_process_group(
        backend=backend,
        rank=rank,
        world_size=world_size
    )


def destroy_distributed_env():
    """
    销毁进程组并清理资源
    """
    dist.destroy_process_group()

    if torch.cuda.is_available():
        torch.cuda.empty_cache()


# ============================
# 三、DDP 训练主逻辑（教学重点）
# ============================
def ddp_training_worker(rank: int, world_size: int, backend: str):
    """
    每一个进程都会执行这个函数
    rank      : 当前进程编号
    world_size: 总进程数
    """

    # ------------------------------------------------
    # 1. 固定随机种子（确保所有进程行为一致）
    # ------------------------------------------------
    seed = 42
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)

    # 确保 cudnn 的确定性行为（教学更容易对齐结果）
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # ------------------------------------------------
    # 2. 初始化分布式环境
    # ------------------------------------------------
    init_distributed_env(rank, world_size, backend)

    # ------------------------------------------------
    # 3. 为当前进程分配设备
    # ------------------------------------------------
    if torch.cuda.is_available():
        torch.cuda.set_device(rank)
        device = torch.device(f"cuda:{rank}")
        print(f"[进程 {rank}] 使用 GPU：{device}")
    else:
        device = torch.device("cpu")
        print(f"[进程 {rank}] CUDA 不可用，使用 CPU")

    # ------------------------------------------------
    # 4. 数据预处理与加载
    # ------------------------------------------------
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)),
        # 将 28×28 展平为 784
        transforms.Lambda(lambda x: x.view(-1))
    ])

    full_dataset = datasets.MNIST(
        root="../data",
        train=True,
        download=True,
        transform=transform
    )

    # ------------------------------------------------
    # 5. 手动划分数据（教学版 DDP）
    #    每个进程只处理数据集的一部分
    # ------------------------------------------------
    total_samples = len(full_dataset)
    samples_per_rank = total_samples // world_size

    start_index = rank * samples_per_rank
    end_index = start_index + samples_per_rank

    subset_dataset = torch.utils.data.Subset(
        full_dataset,
        range(start_index, end_index)
    )

    # DataLoader 使用相同随机种子，确保 shuffle 行为可复现
    data_generator = torch.Generator()
    data_generator.manual_seed(seed)

    train_loader = torch.utils.data.DataLoader(
        subset_dataset,
        batch_size=64,
        shuffle=True,
        generator=data_generator
    )

    print(
        f"[进程 {rank}] 负责样本区间："
        f"{start_index} ~ {end_index - 1}，"
        f"共 {samples_per_rank} 条数据"
    )

    # ------------------------------------------------
    # 6. 创建模型与优化器
    # ------------------------------------------------
    model = SimpleNet(
        input_dim=784,
        hidden_dim=50,
        num_classes=10
    ).to(device)

    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    # ------------------------------------------------
    # 7. 步骤一：广播模型参数（关键教学点）
    #    确保所有进程从“完全相同”的初始模型开始
    # ------------------------------------------------
    for param in model.parameters():
        dist.broadcast(param.data, src=0)

    model.train()

    # ------------------------------------------------
    # 8. 正式开始训练
    # ------------------------------------------------
    for epoch in range(1, 3):
        for batch_idx, (data, target) in enumerate(train_loader):
            data = data.to(device)
            target = target.to(device)

            # ----------------------------
            # 步骤二：前向 + 反向传播
            # 每个进程只计算自己那一部分数据的梯度
            # ----------------------------
            optimizer.zero_grad()

            output = model(data)
            loss = F.nll_loss(output, target)

            step_start_time = time.time()
            loss.backward()

            # ----------------------------
            # 步骤三：梯度 All-Reduce
            # 将所有进程的梯度相加，再取平均
            # ----------------------------
            comm_start_time = time.time()

            for param in model.parameters():
                if param.grad is not None:
                    dist.all_reduce(
                        param.grad.data,
                        op=dist.ReduceOp.SUM
                    )
                    param.grad.data /= world_size

            comm_time = time.time() - comm_start_time

            # ----------------------------
            # 步骤四：参数更新
            # 所有进程使用“完全相同”的平均梯度
            # ----------------------------
            optimizer.step()

            step_time = time.time() - step_start_time

            if batch_idx % 50 == 0:
                print(
                    f"[进程 {rank}] "
                    f"Epoch {epoch} | "
                    f"Batch {batch_idx} | "
                    f"Loss = {loss.item():.6f}"
                )

            if rank == 0 and batch_idx % 50 == 0:
                print(
                    f"[主进程] 单步耗时：{step_time * 1000:.2f} ms，"
                    f"通信耗时：{comm_time * 1000:.2f} ms"
                )

    # ------------------------------------------------
    # 9. 仅在 rank 0 保存模型
    # ------------------------------------------------
    if rank == 0:
        torch.save(model.state_dict(), "mnist_simple_ddp.pt")
        print("[主进程] 模型已保存：mnist_simple_ddp.pt")

    destroy_distributed_env()


# ============================
# 四、程序入口
# ============================
def main():
    # world_size = 使用的进程（GPU）数量
    # 如果只有一张 GPU，就设为 1（等价于普通训练）
    world_size = 1

    # NCCL：GPU 通信首选后端
    backend = "nccl"

    # 检查数据是否能被平均分配
    dataset_size = len(
        datasets.MNIST("../data", train=True, download=True)
    )

    if dataset_size % world_size != 0:
        print(
            f"警告：数据量 {dataset_size} 不能被 world_size={world_size} 整除，"
            f"将丢弃部分样本以保证均分"
        )

    print(f"启动分布式训练，进程数：{world_size}")
    print(f"每个进程处理样本数：{dataset_size // world_size}")

    # 启动多进程
    spawn(
        ddp_training_worker,
        args=(world_size, backend),
        nprocs=world_size,
        join=True
    )

    print("分布式训练结束")


if __name__ == "__main__":
    main()


Overwriting ddp_model_demo.py


In [2]:
!python ddp_model_demo.py

启动分布式训练，进程数：1
每个进程处理样本数：60000
[进程 0] 使用 GPU：cuda:0
[进程 0] 负责样本区间：0 ~ 59999，共 60000 条数据
[进程 0] Epoch 1 | Batch 0 | Loss = 2.294834
[主进程] 单步耗时：178.22 ms，通信耗时：14.50 ms
[进程 0] Epoch 1 | Batch 50 | Loss = 0.515532
[主进程] 单步耗时：1.15 ms，通信耗时：0.30 ms
[进程 0] Epoch 1 | Batch 100 | Loss = 0.286108
[主进程] 单步耗时：1.21 ms，通信耗时：0.32 ms
[进程 0] Epoch 1 | Batch 150 | Loss = 0.235657
[主进程] 单步耗时：1.63 ms，通信耗时：0.49 ms
[进程 0] Epoch 1 | Batch 200 | Loss = 0.305218
[主进程] 单步耗时：1.06 ms，通信耗时：0.29 ms
[进程 0] Epoch 1 | Batch 250 | Loss = 0.421952
[主进程] 单步耗时：1.36 ms，通信耗时：0.30 ms
[进程 0] Epoch 1 | Batch 300 | Loss = 0.376283
[主进程] 单步耗时：1.17 ms，通信耗时：0.32 ms
[进程 0] Epoch 1 | Batch 350 | Loss = 0.159866
[主进程] 单步耗时：1.60 ms，通信耗时：0.39 ms
[进程 0] Epoch 1 | Batch 400 | Loss = 0.211988
[主进程] 单步耗时：1.91 ms，通信耗时：0.65 ms
[进程 0] Epoch 1 | Batch 450 | Loss = 0.401293
[主进程] 单步耗时：1.24 ms，通信耗时：0.32 ms
[进程 0] Epoch 1 | Batch 500 | Loss = 0.201109
[主进程] 单步耗时：1.12 ms，通信耗时：0.31 ms
[进程 0] Epoch 1 | Batch 550 | Loss = 0.257683
[主进程] 单步耗时：1.62 ms，通信

# 问题（minimal_ddp_flat_benchmarking）：2分

修改您的最小DDP实现，以从所有参数中传递一个扁平化的梯度张量。将其性能与在先前使用条件下（1个节点 x 2个GPU，XL模型大小如§1.1.3中所述）为每个参数张量发出一个all-reduce的最小DDP实现进行比较。

交付物：在单次批量all-reduce调用下，未分布式数据并行训练中每个训练迭代的测量时间以及梯度通信所花费的时间。1-2句话比较批量与单独通信梯度的结果。

**验证：把“很多小的梯度通信”合并成“一次大的梯度通信”，能否显著降低 DDP 的通信开销**

这在真实系统里是一个**非常核心的工程问题**。



## naive DDP （对照基线）

之前的最小 DDP 实现是这样的逻辑：

```text
for param in model.parameters():
    all_reduce(param.grad)
```

也就是：**每个参数张量**，**单独一次 all-reduce**，通信调用次数 = 参数个数，XL 模型来说：参数张量数量几百～上千个，NCCL / GPU 通信，每一次 all-reduce 都有 **启动开销（latency）**，小 tensor → 极其低效，这就是作业的**优化的瓶颈**

---

## 改什么

###  核心改动：**梯度展平 + 单次通信**

把：

```python
grad_1, grad_2, grad_3, ..., grad_n
```

变成：

```python
flat_grad = concat([grad_1, grad_2, ..., grad_n])
```

然后只做：

```python
dist.all_reduce(flat_grad)
```

再把结果 **切回每个参数的 grad**



In [7]:
%%writefile minimal_ddp_flat_benchmarking.py
import torch
import torch.distributed as dist
import os
from torchvision import datasets, transforms
import torch.optim as optim
import time
from torch.multiprocessing.spawn import spawn
import numpy as np
import random
import torch.nn as nn
import torch.nn.functional as F

class SimpleNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SimpleNet, self).__init__()
        # 定义第一个隐藏层
        self.fc1 = nn.Linear(input_size, hidden_size)
        # 定义第二个隐藏层
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        # 定义输出层
        self.fc3 = nn.Linear(hidden_size, num_classes)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)  
# 示例：创建一个SimpleNet实例

def setup(rank, world_size, backend):
    """ 初始化分布式环境 """
    os.environ['MASTER_ADDR'] = 'localhost' # 设置主节点IP地址以及端口，其他节点需要通过这个地址连接到主节点
    os.environ['MASTER_PORT'] = '29500'
    # 根据后端初始化进程组
    dist.init_process_group(backend, rank=rank, world_size=world_size) # 初始化进程组，rank是当前进程的rank，world_size是总进程数，backend: 这是指定通信后端的参数。常见的后端有：gloo(CPU), nccl(GPU), mpi等。

def cleanup():
    """ 清理分布式环境 """
    dist.destroy_process_group()
    # 清理GPU缓存
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

def ddp_train(rank, world_size, backend):
    """ Naive DDP训练函数，实现图片中描述的4个步骤 """
    # 设置随机种子确保可重现性（所有进程使用相同种子）
    seed = 42
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    
    # 设置确定性行为
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # 初始化分布式环境
    setup(rank, world_size, backend)
    
    # 为每个进程分配不同的GPU设备
    # 使用GPU 0和1，它们当前是空闲的
    gpu_id = rank  # rank 0 -> GPU 0, rank 1 -> GPU 1
    if torch.cuda.is_available():
        torch.cuda.set_device(gpu_id)
        device = torch.device(f"cuda:{gpu_id}")
        print(f"Rank {rank} using GPU: {device}")
    else:
        device = torch.device("cpu")
        print(f"CUDA not available, using CPU for rank {rank}")
    
    # 数据加载和预处理
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)),
        transforms.Lambda(lambda x: x.view(-1))
    ])
    
    dataset = datasets.MNIST('../data', train=True, download=True, transform=transform)
    
    # 为了验证DDP正确性，我们让每个进程处理不同的数据子集
    # 这样总的有效batch size = 64 * world_size
    total_samples = len(dataset)
    samples_per_process = total_samples // world_size
    start_idx = rank * samples_per_process
    end_idx = start_idx + samples_per_process
    
    # 创建当前进程的数据子集
    subset = torch.utils.data.Subset(dataset, range(start_idx, end_idx))
    
    # 设置确定性的数据加载器
    # 重要：为了与单进程训练比较，我们需要确保数据顺序一致
    generator = torch.Generator()
    generator.manual_seed(seed)  # 所有进程使用相同的种子确保一致性
    train_loader = torch.utils.data.DataLoader(subset, batch_size=64, shuffle=True, generator=generator)
    
    print(f"Rank {rank} processing samples {start_idx} to {end_idx-1} ({samples_per_process} samples)")
    print(f"Rank {rank} effective batch size: 64, total distributed batch size: {64 * world_size}")
    
    # 创建模型和优化器（每个设备都创建相同的模型）
    model = SimpleNet(input_size=784, hidden_size=50, num_classes=10).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    # ===== 新增：记录梯度展平所需的元信息 =====
    grad_shapes = []
    grad_numels = []

    for p in model.parameters():
        grad_shapes.append(p.shape)
        grad_numels.append(p.numel())

    total_grad_numel = sum(grad_numels)

    # 步骤1: Broadcast模型参数从rank 0到所有其他ranks
    # 确保所有设备从相同的初始模型和优化器状态开始
    for param in model.parameters():
        # 确保参数在正确的设备上进行broadcast
        dist.broadcast(param.data, src=0)
    
    # 同步优化器状态（如果有的话）
    for group in optimizer.param_groups:
        for param in group['params']:
            if param in optimizer.state:
                for key, value in optimizer.state[param].items():
                    if torch.is_tensor(value):
                        dist.broadcast(value, src=0)
    
    model.train()
    
    for epoch in range(1, 3):  # Train for 2 epochs
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            
            # 步骤2: 每个设备使用本地模型参数进行前向传播和反向传播
            # 计算n/d个样本的梯度
            optimizer.zero_grad()
            output = model(data)
            loss = torch.nn.functional.nll_loss(output, target)
            step_start = time.time()
            loss.backward()
            
            comm_start = time.time()
            # 步骤3: All-reduce梯度
            # 将所有设备的梯度求平均，使每个设备都持有所有n个样本的平均梯度

            # ===== 新的步骤3：Flattened gradient all-reduce =====
            flat_grad = torch.zeros(total_grad_numel, device=device)

            offset = 0
            for p in model.parameters():
                if p.grad is not None:
                    numel = p.grad.numel()
                    flat_grad[offset:offset + numel] = p.grad.view(-1)
                    offset += numel

            # 单次通信
            dist.all_reduce(flat_grad, op=dist.ReduceOp.SUM)
            flat_grad /= world_size

            # 写回每个参数的梯度
            offset = 0
            for p, shape, numel in zip(model.parameters(), grad_shapes, grad_numels):
                if p.grad is not None:
                    p.grad.copy_(
                        flat_grad[offset:offset + numel].view(shape)
                    )
                    offset += numel

            comm_time = time.time() - comm_start
            # 步骤4: 优化器步骤 - 每个设备使用相同的平均梯度更新参数
            # 由于所有设备从相同初始状态开始并使用相同梯度，参数会保持同步
            optimizer.step()
            step_time = time.time() - step_start
            if batch_idx % 50 == 0:
                print(f'Rank {rank}, Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
            if rank == 0 and batch_idx % 50 == 0:
                print(f"Step time: {step_time*1000:.2f} ms | Comm time: {comm_time*1000:.2f} ms")

    # 只在rank 0保存模型
    if rank == 0:
        torch.save(model.state_dict(), "mnist_simple_ddp.pt")
        print("Model saved!")
            
    cleanup()

def main():
    world_size = 1 # gpu只有一个就改成1，但是这样和平时普通的训练就没有区别了
    backend = 'nccl'  # 使用gloo后端，更适合CPU训练
    # 只检查数据集大小是否能被world_size整除
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    
    dataset = datasets.MNIST('../data', train=True, download=True, transform=transform)
    data_size = len(dataset)

    if data_size % world_size != 0:
        print(f"Warning: Data size {data_size} is not divisible by world size {world_size}")
        print(f"Some samples will be ignored to ensure equal distribution")
    
    print(f"Starting distributed training with {world_size} processes")
    print(f"Each process will handle {data_size // world_size} samples")
    print(f"Total samples: {data_size}")
    
    # 启动分布式训练
    spawn(ddp_train, args=(world_size, backend), nprocs=world_size, join=True)
    
    print("Distributed training completed!")

if __name__ == '__main__':
    main()

Overwriting minimal_ddp_flat_benchmarking.py


In [8]:
!python minimal_ddp_flat_benchmarking.py


Traceback (most recent call last):
  File "/mnt/d/code/项目/cs336/CS336-Chinese-co-construction/coursework/Assignment2_System/cs336_systems/minimal_ddp_flat_benchmarking.py", line 4, in <module>
    from torchvision import datasets, transforms
ModuleNotFoundError: No module named 'torchvision'


# 作业三

## 问题 (ddp_overlap_individual_parameters): 5 分

实现一个 Python 类来处理分布式数据并行训练。该类应包装任意 PyTorch `nn.Module` 并在训练前广播权重（以便所有等级具有相同的初始参数），并发出通信调用以进行梯度平均。我们建议以下公共接口：

```python
def __init__(self, module: torch.nn.Module): 给定一个实例化的 PyTorch nn.Module 要并行化，构建一个 DDP 容器，该容器将处理跨等级的梯度同步。

def forward(self, *inputs, **kwargs): 调用包装模块的 forward() 方法，并使用提供的定位参数和关键字参数。

def finish_gradient_synchronization(self): 当被调用时，等待异步通信
```

*在高级情况下，如果你使用多个 CUDA 流，你可能需要显式同步跨流以确保输出准备好进行后续操作。* 参见 [pytorch.org/docs/stable/notes/cuda.html#cuda-streams](https://pytorch.org/docs/stable/notes/cuda.html#cuda-streams)。

### GPU 上的调用

为了使用此类执行分布式训练，我们将一个模块传递给该模块进行包装，然后在我们运行 `optimizer.step()` 之前添加一个对 `finish_gradient_synchronization()` 的调用，以确保依赖于梯度的优化器步骤可以排队：

```python
model = ToyModel().to(device)
ddp_model = DDP(model)

for _ in range(train_steps):
    x, y = get_batch()
    logits = ddp_model(x)
    loss = loss_fn(logits, y)
    loss.backward()
    ddp_model.finish_gradient_synchronization()
    optimizer.step()
```

### 交付物：实现一个容器类来处理分布式数据并行训练。该类应重叠梯度通信和反向传播的计算。为了测试你的 DDP 类，首先实现适配器 [adapters.get_ddp_individual_parameters](https://pytorch.org/docs/stable/notes/cuda.html#cuda-streams) 和 [adapters.ddp_individual_parameters_on_after_backward](https://pytorch.org/docs/stable/notes/cuda.html#cuda-streams)（后者是可选的，取决于你的实现，你可能不需要它）。

然后，执行测试，通过运行 `pytest tests/test_ddp_individual_parameters.py`。我们建议多次运行测试（例如，5 次）以确保其可靠通过。

以下是翻译和整理为 Markdown 格式的内容：

---

## 问题 (ddp_overlap_individual_parameters): 5 分

实现一个 Python 类来处理分布式数据并行训练。该类应包装一个任意的 PyTorch `nn.Module` 并在训练之前处理权重的广播（以便所有 ranks 具有相同的初始参数）以及发出通信调用以进行梯度平均。我们建议以下公共接口：

```python
def __init__(self, module: torch.nn.Module): 给定一个实例化的 PyTorch nn.Module 要并行化，构建一个 DDP 容器，该容器将处理跨 ranks 的梯度同步。

def forward(self, *inputs, **kwargs): 调用被包装模块的 `forward()` 方法，并使用提供的定位参数和关键字参数。

def finish_gradient_synchronization(self): 当调用时，等待异步通信
```

*在高级情况下，如果你使用多个 CUDA 流，你可能需要显式同步跨流以确保输出准备好进行后续操作。* 参见 [CUDA Streams](https://pytorch.org/docs/stable/notes/cuda.html#cuda-streams)。

### GPU 上的调用

为了使用此类执行分布式训练，我们将一个模块传递给该模块进行包装，然后在我们运行 `optimizer.step()` 之前添加一个对 `finish_gradient_synchronization()` 的调用，以确保依赖于梯度的优化器步骤可以排队：

```python
model = ToyModel().to(device)
ddp_model = DDP(model)

for _ in range(train_steps):
    x, y = get_batch()
    logits = ddp_model(x)
    loss = loss_fn(logits, y)
    loss.backward()
    ddp_model.finish_gradient_synchronization()
    optimizer.step()
```

### 交付物：实现一个容器类来处理分布式数据并行训练。该类应重叠梯度通信和反向传播的计算。为了测试你的 DDP 类，首先实现适配器 `adapters.get_ddp_individual_parameters` 和 `adapters.ddp_individual_parameters_on_after_backward`（后者是可选的，取决于你的实现，你可能不需要它）。

然后，执行测试，通过运行 `pytest tests/test_ddp_individual_parameters.py`。我们建议多次运行测试（例如，5 次）以确保其可靠通过。

---

In [ ]:
%%writefile ddp_overlap_individual_parameters.py
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist
import os



# ============================
# 二、手动 DDP（按桶通信梯度）
# ============================
class DDPOverlapBucketed(nn.Module):
    def __init__(self, module: torch.nn.Module, bucket_size_mb: float):
        super(DDPOverlapBucketed, self).__init__()
        self.module = module
        self.bucket_size_bytes = bucket_size_mb * 1024 * 1024
        self.handles = []   # 保存异步 all_reduce 句柄
        self.buckets = []   # 梯度桶
        self.world_size = dist.get_world_size()
        
        # --------------------
        # 初始化：广播参数 + 创建桶 + 注册钩子
        # --------------------
        self._broadcast_parameters()  # 初始参数广播
        self._create_bucket()         # 按大小划分梯度桶
        self._register_hook()         # 注册梯度钩子实现异步通信
    
    # --------------------
    # 广播初始参数
    # --------------------
    def _broadcast_parameters(self):
        """
        将 rank 0 的参数广播到所有进程，确保初始状态一致
        """
        if self.world_size > 1:
            for param in self.module.parameters():
                dist.broadcast(param.data, src=0)
    
    # --------------------
    # 按 bucket_size 创建梯度桶
    # --------------------
    def _create_bucket(self):
        current_bucket_size = 0
        current_bucket = []

        # 倒序遍历参数，构建桶
        for p in reversed(list(self.module.parameters())):
            if p.requires_grad:
                p_size = p.numel() * p.element_size()
                
                # 当前桶满了则保存并创建新桶
                if p_size + current_bucket_size > self.bucket_size_bytes and current_bucket:
                    self.buckets.append(current_bucket)
                    current_bucket_size = 0
                    current_bucket = []
                
                current_bucket.append(p)
                current_bucket_size += p_size
        
        # 如果还有剩余参数，作为最后一个桶
        if current_bucket:
            self.buckets.append(current_bucket)
        
        # 为每个桶创建缓冲区
        for i, bucket_params in enumerate(self.buckets):
            if not bucket_params:
                continue
            buffer_size = sum(p.numel() for p in bucket_params)
            buffer = torch.zeros(buffer_size, device=bucket_params[0].device, dtype=bucket_params[0].dtype)
            self.buckets[i] = {
                "params": bucket_params,
                "buffer": buffer,
                "ready_params": set(),
                "triggered": False
            }
    
    # --------------------
    # 注册梯度钩子
    # --------------------
    def _register_hook(self):
        for bucket_idx, bucket_info in enumerate(self.buckets):
            for param in bucket_info["params"]:
                # 闭包捕获当前 bucket_idx
                def make_hook(idx):
                    return lambda grad, param=param: self._create_hook(grad, param, idx)
                param.register_hook(make_hook(bucket_idx))
    
    # --------------------
    # 梯度钩子逻辑：延迟执行 all_reduce
    # --------------------
    def _create_hook(self, grad, param, bucket_idx):
        bucket_info = self.buckets[bucket_idx]
        bucket_info["ready_params"].add(param)

        # 当桶内所有参数梯度都准备好，且未触发通信
        if len(bucket_info["ready_params"]) == len(bucket_info["params"]) and not bucket_info["triggered"]:
            bucket_info["triggered"] = True  # 标记为已触发

            def delayed_sync():
                # 将桶内所有梯度拷贝到扁平缓冲区
                offset = 0
                for p in bucket_info["params"]:
                    numel = p.numel()
                    if p.grad is not None:
                        bucket_info["buffer"][offset:offset+numel].copy_(p.grad.view(-1))
                    else:
                        bucket_info["buffer"][offset:offset+numel].zero_()
                    offset += numel
                
                # 启动异步 all_reduce，实现计算与通信重叠
                handle = dist.all_reduce(bucket_info["buffer"], async_op=True)
                self.handles.append((handle, bucket_idx))
            
            # 延迟执行，确保所有梯度计算完成
            import torch.autograd as autograd
            autograd.Variable._execution_engine.queue_callback(delayed_sync)
    
    # --------------------
    # forward 前清理状态
    # --------------------
    def forward(self, x):
        if self.world_size > 1:
            for bucket in self.buckets:
                bucket["triggered"] = False
                bucket["ready_params"].clear()
        self.handles.clear()
        return self.module(x)

    # --------------------
    # 等待所有异步通信完成
    # --------------------
    def finish_gradient_synchronization(self):
        """
        等待所有 all_reduce 完成，并将梯度写回各参数
        需在 optimizer.step() 之前调用
        """
        for handle, bucket_idx in self.handles:
            handle.wait()  # 等待通信完成

            bucket_info = self.buckets[bucket_idx]
            buffer = bucket_info["buffer"]

            # 求平均梯度
            buffer.div_(self.world_size)

            offset = 0
            for p in bucket_info["params"]:
                numel = p.numel()
                if p.grad is not None:
                    p.grad.view(-1).copy_(buffer[offset:offset+numel])
                offset += numel
        
        # 清空 handles，为下一次迭代准备
        self.handles.clear()


# ============================
# 三、简单示例运行
# ============================
def run():
    # 初始化分布式环境（torchrun 已设置环境变量）
    dist.init_process_group(backend="nccl")
    local_rank = int(os.environ["LOCAL_RANK"])
    torch.cuda.set_device(local_rank)

    model = nn.Linear(10, 5).cuda()
    ddp_model = DDPOverlapBucketed(model, bucket_size_mb=1)  # 1 MB 小桶方便观察
    opt = torch.optim.SGD(ddp_model.parameters(), lr=0.1)

    for step in range(3):
        x = torch.randn(4, 10).cuda()
        loss = ddp_model(x).sum()
        loss.backward()
        ddp_model.finish_gradient_synchronization()  # 必须手动调用
        opt.step()
        opt.zero_grad()
        print(f"[rank {dist.get_rank()}] step {step} loss={loss.item()}")


if __name__ == "__main__":
    run()


Overwriting ddp_overlap_individual_parameters.py


In [7]:
!torchrun --nproc_per_node=1 ddp_overlap_individual_parameters.py # 有多少张卡设置nproc_per_node=几

[rank 0] step 0 loss=2.803687572479248
[rank 0] step 1 loss=-8.734395980834961
[rank 0] step 2 loss=-22.41156768798828
[rank0]:[W122 14:35:46.682734635 ProcessGroupNCCL.cpp:1524] Warning: WARNING: destroy_process_group() was not called before program exit, which can leak resources. For more info, please see https://pytorch.org/docs/stable/distributed.html#shutdown (function operator())


In [11]:

%%writefile distributed_communication_single_node_demo.py
import os
import time
import torch
import torch.distributed as dist
import argparse
import torch
from torch.multiprocessing import spawn


# ============================================================
# 1. 分布式环境初始化与清理
# ============================================================

def init_distributed_environment(
    master_addr: str,
    master_port: int,
    global_rank: int,
    world_size: int,
    backend: str
):
    """
    初始化 PyTorch 分布式通信环境（进程组）

    参数说明：
    - master_addr : rank 0 所在节点的 IP 地址
    - master_port : 用于进程间通信的端口
    - global_rank : 当前进程在所有进程中的唯一编号
    - world_size  : 总进程数
    - backend     : 通信后端（gloo / nccl / mpi）
    """

    # --------------------------------------------------------
    # 1.1 设置 rendezvous（所有进程汇合的地址）
    # --------------------------------------------------------
    # 所有进程必须通过 MASTER_ADDR:MASTER_PORT 建立初始连接
    os.environ["MASTER_ADDR"] = master_addr
    os.environ["MASTER_PORT"] = str(master_port)

    # --------------------------------------------------------
    # 1.2 初始化进程组（分布式的“总开关”）
    # --------------------------------------------------------
    # 这是使用 torch.distributed 的前置条件
    dist.init_process_group(
        backend=backend,
        rank=global_rank,
        world_size=world_size
    )


def destroy_distributed_environment():
    """
    清理分布式环境并释放资源
    """

    # 销毁进程组，防止资源泄漏或死锁
    dist.destroy_process_group()

    # GPU 场景下，清空 PyTorch 的 CUDA cache（非强制，但推荐）
    if torch.cuda.is_available():
        torch.cuda.empty_cache()


# ============================================================
# 2. All-Reduce 通信性能 Benchmark
# ============================================================

def run_all_reduce_benchmark(
    global_rank: int,
    world_size: int,
    tensor_size_mb: int,
    backend: str,
    device: str,
    master_addr: str,
    master_port: int
):
    """
    对 dist.all_reduce 进行性能测试

    测试指标：
    - 单次 all-reduce 平均耗时
    - 理论通信带宽（GB/s）
    """

    # --------------------------------------------------------
    # 2.1 初始化分布式环境
    # --------------------------------------------------------
    init_distributed_environment(
        master_addr=master_addr,
        master_port=master_port,
        global_rank=global_rank,
        world_size=world_size,
        backend=backend
    )

    # --------------------------------------------------------
    # 2.2 绑定 GPU（仅在 CUDA 场景下）
    # --------------------------------------------------------
    if device == "cuda":
        # 通常约定：rank i → GPU i
        torch.cuda.set_device(global_rank)
        torch.cuda.empty_cache()

    # --------------------------------------------------------
    # 2.3 构造通信测试用的 Tensor
    # --------------------------------------------------------
    # 将 MB 转换为字节
    tensor_num_bytes = tensor_size_mb * 1024 * 1024

    # float32 占 4 字节
    bytes_per_element = 4
    num_elements = tensor_num_bytes // bytes_per_element

    # 随机生成测试数据（数值本身不重要）
    communication_tensor = torch.randn(
        num_elements,
        device=device,
        dtype=torch.float32
    )

    # --------------------------------------------------------
    # 2.4 Warm-up（非常重要）
    # --------------------------------------------------------
    # 原因：
    # - NCCL 通信器初始化
    # - CUDA kernel lazy initialization
    # - GPU 频率爬升
    warmup_iterations = 5
    for _ in range(warmup_iterations):
        dist.all_reduce(
            communication_tensor,
            op=dist.ReduceOp.SUM
        )

        # CUDA 是异步执行，必须显式同步
        if device == "cuda":
            torch.cuda.synchronize()

    # 确保所有进程在同一时刻开始正式测试
    dist.barrier()

    # --------------------------------------------------------
    # 2.5 正式 benchmark（计时）
    # --------------------------------------------------------
    benchmark_iterations = 20

    start_time = time.time()

    for _ in range(benchmark_iterations):
        dist.all_reduce(
            communication_tensor,
            op=dist.ReduceOp.SUM
        )

    # GPU 场景下等待所有 kernel 完成
    if device == "cuda":
        torch.cuda.synchronize()

    end_time = time.time()

    # --------------------------------------------------------
    # 2.6 性能指标计算
    # --------------------------------------------------------
    total_elapsed_time = end_time - start_time
    avg_latency_seconds = total_elapsed_time / benchmark_iterations

    # 带宽 = 数据量 / 时间（单位：GB/s）
    bandwidth_gbps = (tensor_num_bytes / avg_latency_seconds) / 1e9

    # --------------------------------------------------------
    # 2.7 打印结果（仅 rank 0）
    # --------------------------------------------------------
    if global_rank == 0:
        print(
            f"[All-Reduce Benchmark]\n"
            f"  Backend        : {backend}\n"
            f"  Device         : {device}\n"
            f"  World Size     : {world_size}\n"
            f"  Tensor Size    : {tensor_size_mb} MB\n"
            f"  Avg Latency    : {avg_latency_seconds * 1000:.4f} ms\n"
            f"  Bandwidth      : {bandwidth_gbps:.4f} GB/s\n"
        )

    # --------------------------------------------------------
    # 2.8 汇总所有 rank 的结果（教学用）
    # --------------------------------------------------------
    local_result = {
        "rank": global_rank,
        "world_size": world_size,
        "backend": backend,
        "device": device,
        "tensor_size_mb": tensor_size_mb,
        "avg_latency_ms": avg_latency_seconds * 1000,
        "bandwidth_gbps": bandwidth_gbps,
    }

    gathered_results = [None for _ in range(world_size)]
    dist.all_gather_object(gathered_results, local_result)

    # --------------------------------------------------------
    # 2.9 清理环境
    # --------------------------------------------------------
    destroy_distributed_environment()

    # 只让主进程返回结果
    if global_rank == 0:
        return gathered_results


def main():

    for size in [1,10,100,1000]:
        for Backend in ["gloo", "nccl"]:
            for world_size in [1]:
                parser = argparse.ArgumentParser("All-Reduce Benchmark")

                parser.add_argument("--world_size", type=int, default=world_size,
                                    help="进程总数（通常等于 GPU 数）")
                parser.add_argument("--tensor_size_mb", type=int, default=size,
                                    help="通信 tensor 大小（MB）")
                parser.add_argument("--backend", type=str, default=Backend,
                                    choices=["gloo", "nccl"],
                                    help="分布式通信后端")
                parser.add_argument("--device", type=str, default="cuda",
                                    choices=["cpu", "cuda"],
                                    help="运行设备")
                parser.add_argument("--master_addr", type=str, default="127.0.0.1")
                parser.add_argument("--master_port", type=int, default=29500)

                args = parser.parse_args()

                # GPU 数量检查
                if args.device == "cuda":
                    assert torch.cuda.is_available(), "CUDA 不可用"
                    assert args.world_size <= torch.cuda.device_count(), (
                        "world_size 不能超过 GPU 数量"
                    )

                # 使用 spawn 启动多进程
                spawn(
                    fn=run_all_reduce_benchmark,
                    args=(
                        args.world_size,
                        args.tensor_size_mb,
                        args.backend,
                        args.device,
                        args.master_addr,
                        args.master_port
                    ),
                    nprocs=args.world_size,
                    join=True
                )


if __name__ == "__main__":
    main()



Overwriting distributed_communication_single_node_demo.py


In [14]:
!python distributed_communication_single_node_demo.py

[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[All-Reduce Benchmark]
  Backend        : gloo
  Device         : cuda
  World Size     : 1
  Tensor Size    : 1 MB
  Avg Latency    : 0.2266 ms
  Bandwidth      : 4.6281 GB/s

/mnt/d/code/项目/cs336/CS336-Chinese-co-construction/coursework/Assignment2_System/.venv/lib/python3.12/site-packages/torch/distributed/distributed_c10d.py:4876: UserWarning: barrier(): using the device under current context. You can specify `device_id` in `init_process_group` to mute this warning.
  warnings.warn(  # warn only once
[All-Reduce Benchmark]
  Backend        : nccl
  Device         : cuda
  World Size     : 1
  Tensor Size    : 1 MB
  Avg Latency    : 0.0165 ms
  Bandwidth      : 63.7398 GB/s

[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[All-Reduce Benchmark]
  Backend        : gloo
  Device         : cuda
  World Size     : 1
  Tensor Size    : 10 MB
  Avg Latency

# 实现分布式优化器封装

实现了一个**参数分片（sharding）的分布式优化器封装器**，思想上接近 **ZeRO Stage-1 / FSDP 的 optimizer state sharding**

## 一、整体功能概览

`ShardedOptimizer` 的目标是：

> **在数据并行（DDP）场景下，让不同 rank 只负责一部分参数的 optimizer state 和更新计算，从而节省显存。**

核心思想是对于**参数本身，每个 rank 都有完整模型参数**，对于**优化器状态（momentum / Adam 的 exp_avg 等），只在参数 owner rank 上存在**， **参数更新只在 owner rank 上进行**。
* **参数同步：更新后通过 `broadcast` 把新参数发给其他 rank**。

## 二、整体架构

```
所有 rank：
  拥有完整模型参数
       │
       ▼
参数被 round-robin 分配 owner_rank
       │
       ▼
每个 rank：
  ├─ 全局 Optimizer（只用于 param_groups 管理）
  └─ 本地 optim（只包含自己拥有的 params）
       │
       ▼
step():
  1. 本地 optim.step()（只更新自己那一片）
  2. dist.broadcast（把更新后的参数同步给所有 rank）
```

---

## 三、代码解读


### 1。 初始化：`__init__`

```python
class ShardedOptimizer(Optimizer):
```

继承 PyTorch 原生 `Optimizer`，因此：

这样可以被 Trainer / 训练循环无缝使用，并且拥有 `param_groups / zero_grad / step` 接口

---

#### 1.1 记录原始 optimizer 类型

```python
self.optimizer_cls = optimizer_cls
self.optimizer_kwargs = kwargs
```

例如：

```python
ShardedOptimizer(
    model.parameters(),
    torch.optim.Adam,
    lr=1e-4
)
```

---

#### 1.2 获取分布式信息

```python
self.rank = dist.get_rank()
self.world_size = dist.get_world_size()
```

用于：

* 决定参数 owner
* 决定 broadcast 源

---

#### 1.3 参数分配辅助变量

```python
self.global_param_counter = 0
self.local_param_groups = []
```

* `global_param_counter`：**全局参数编号**
* 用 round-robin 把参数均匀分配给各个 rank

---

#### 1.4 调用父类构造函数（关键）

```python
super().__init__(params, defaults=kwargs)
```


`Optimizer.__init__` 内部会调用 `self.add_param_group`，但这里的 `add_param_group` 已经被 **重写**

这使得 **参数分片逻辑嵌入在初始化阶段**

---

#### 1.5 创建“本地 optimizer”

```python
self.optim = self.optimizer_cls(self.local_param_groups, **self.optimizer_kwargs)
```

`self.optim` **只包含当前 rank 拥有的参数**，只有这些参数才会分配 optimizer state（节省显存）。

---

### 2. 参数分片逻辑：`add_param_group`

这是整个实现的**核心函数**。

---

#### 2.1 给每个参数打 owner 标记

```python
p._owner_rank = self.global_param_counter % self.world_size
```

round-robin 分配, `_owner_rank` 是动态属性（hack，但可行）

示例（world_size=4）：

| 参数编号 | owner_rank |
| ---- | ---------- |
| p0   | 0          |
| p1   | 1          |
| p2   | 2          |
| p3   | 3          |
| p4   | 0          |
| p5   | 1          |

---

#### 2.2 加入“全局 optimizer”

```python
super().add_param_group(param_group)
```

**为什么必须这么做？**

`self.param_groups` 需要包含 **所有参数**,用于进行`zero_grad`，遍历所有参数做 `broadcast`，并且Trainer / AMP 兼容，**注意**：这个“全局 optimizer”**不做 step**。

---

#### 2.3 构造本地 param group

```python
local_group['params'] = [
    p for p in param_group['params']
    if p._owner_rank == self.rank
]
```

它只保留属于当前 rank 的参数，其他超参（lr、weight_decay）保持一致。

---

#### 2.4 加入本地 optimizer

```python
self.optim.add_param_group(local_group)
```

或者（初始化期间）缓存起来：

```python
self.local_param_groups.append(local_group)
```

---

### 3. 参数更新逻辑：`step`

```python
def step(self, closure=None, **kwargs):
```

---

#### Step 1：本地更新

```python
self.optim.step(**kwargs)
```

* **只更新自己拥有的参数**
* 只有这些参数有 optimizer state

---

#### Step 2：参数同步（核心）

```python
dist.broadcast(p.data, src=p._owner_rank)
```

如果是 owner 则发送最新参数，如果不是 owner 就接收参数覆盖本地副本。最终所有 rank 的模型参数 **完全一致**。

---

### 4.梯度清零：`zero_grad`

```python
super().zero_grad(set_to_none=set_to_none)
```

清除所有参数的梯度，即使不是 owner，也要清 grad（否则梯度会累积）。

---

## 四、这个实现“像什么”？

| 技术           | 相似度  | 区别                        |
| ------------ | ---- | ------------------------- |
| ZeRO Stage-1 | ⭐⭐⭐⭐ | 真实 ZeRO 会用 reduce-scatter |
| FSDP         | ⭐⭐   | 没有参数 flatten / overlap    |
| DDP          | ⭐    | DDP 是全量 optimizer state   |

---

## 五、优点与限制

###  优点

1. **显存节省**

   * optimizer state 约减少到 `1/world_size`
2. **逻辑清晰**

   * 教学 & 原理验证非常好
3. **兼容任意 Optimizer**

   * Adam / SGD / Adafactor 等

---

### 局限

1. **通信成本高**

   ```python
   dist.broadcast(p.data)
   ```

   * 每个参数一次 broadcast（非常慢）

2. **参数粒度过细**

   * 实际系统会 **flatten 后按 bucket 同步**

3. **没有 overlap**

   * 计算与通信完全串行

4. **依赖 monkey-patch 参数属性**

   * `_owner_rank` 不够“干净”

---

## 六、总结


> 用 round-robin 分配参数 owner，
> 只在 owner rank 上更新参数，
> 再用 broadcast 同步权重，
> 从而显著降低 optimizer 显存占用。



命令行参数
   ↓
   
wandb 初始化（实验配置中心）
   ↓

加载 tokenized 数据（pkl）
   ↓

构造 DataLoader（按 context_length 切 batch）
   ↓

构造 Transformer 模型
   ↓

构造 AdamW + Cosine 学习率调度
   ↓

训练循环（epoch → step）
   ↓

验证 / 日志 / checkpoint


In [ ]:
"""
=====================================================
Transformer Language Model Training Script (Teaching Version)
=====================================================

本脚本用于训练一个基于 Transformer 的自回归语言模型，
数据集为 TinyStories（已提前 tokenized 并保存为 pkl）。

主要包含：
1. WandB 实验配置与日志
2. 数据加载（自定义 DataLoader）
3. 模型构建（TransformerModule）
4. 训练循环（AdamW + Cosine LR）
5. 验证与模型 checkpoint
"""

import time
import json
import pickle
import argparse
import timeit

import torch
import wandb




#导入自己的模块
from cs336_myown import (
    DataLoader,
    CosineSchedule,
    CrossEntropyLoss,
    clip_gradient
)

# =====================================================
# 一些全局工具变量
# =====================================================

# 用于区分不同实验的时间戳（用于 wandb 名字 & checkpoint）
TIMESTAMP = time.strftime("%Y%m%d_%H%M%S")


# =====================================================
# WandB 初始化函数
# =====================================================
def init_wandb(
    d_model: int,
    d_ff: int,
    n_layers: int,
    n_heads: int,
    batch_size: int,
    epochs: int,
    train_steps: int
):
    """
    初始化 WandB 实验，并统一管理所有超参数。

    教学重点：
    - WandB 的 config 本质上是“实验的单一事实源（single source of truth）”
    - 后续训练中不应再散落硬编码超参数
    """

    wandb.login()

    run = wandb.init(
        project="cs336_final_train",
        config={
            # -----------------------------
            # Experiment metadata
            # -----------------------------
            "experiment_name": f"tinystories_17M_{TIMESTAMP}",
            "total_tokens_processed": 327_680_000,

            # -----------------------------
            # Dataset & Tokenizer
            # -----------------------------
            "train_data_path": "../data/TinyStoriesV2-GPT4-train.txt",
            "valid_data_path": "../data/TinyStoriesV2-GPT4-valid.txt",
            "encoded_ids_train_path": "./cs336_systems/encoded_ids_train.pkl",
            "encoded_ids_valid_path": "./cs336_systems/encoded_ids_valid.pkl",
            "vocab_path": "vocab.json",
            "merges_path": "merges.txt",

            # -----------------------------
            # Model hyperparameters
            # -----------------------------
            "vocab_size": 10_000,
            "context_length": 256,
            "d_model": d_model,        # Transformer hidden size
            "d_ff": d_ff,              # FFN hidden size
            "n_layers": n_layers,      # Transformer block 数
            "n_heads": n_heads,        # Attention heads
            "rope_theta": 10_000.0,    # RoPE 超参数

            # -----------------------------
            # Training hyperparameters
            # -----------------------------
            "batch_size": batch_size,
            "epochs": epochs,
            "train_steps": train_steps,

            # Learning rate schedule
            "initial_lr": 3e-5,
            "max_learning_rate": 3e-5,
            "min_learning_rate": 1e-5,
            "lr_warmup_steps": 2000,
            "cosine_cycle_iters": 10000,

            # Optimizer (AdamW)
            "weight_decay": 0.1,
            "adam_beta1": 0.9,
            "adam_beta2": 0.95,
            "eps": 1e-8,

            # Gradient clipping
            "grad_clip": 1.0,

            # -----------------------------
            # Logging & Checkpoint
            # -----------------------------
            "log_interval": 20,
            "val_interval": 20,
            "checkpoint_interval": 60,
            "checkpoint_dir": "checkpoints",
        }
    )

    return run.config


# =====================================================
# 主训练函数
# =====================================================
def train_model(config, device):
    """
    Transformer 语言模型训练主逻辑

    教学重点：
    - epoch vs step 的区别
    - global_step 的意义（LR schedule 依赖）
    - 训练 / 验证模式切换
    """

    # -----------------------------
    # 1. 加载 tokenized 数据
    # -----------------------------
    with open(config["encoded_ids_train_path"], "rb") as f:
        train_token_ids = pickle.load(f)

    with open(config["encoded_ids_valid_path"], "rb") as f:
        valid_token_ids = pickle.load(f)

    train_loader = DataLoader(
        train_token_ids,
        batch_size=config["batch_size"],
        context_length=config["context_length"],
        shuffle=True
    )

    valid_loader = DataLoader(
        valid_token_ids,
        batch_size=config["batch_size"],
        context_length=config["context_length"],
        shuffle=True
    )

    # -----------------------------
    # 2. 构建模型
    # -----------------------------
    model = TransformerModule(
        d_model=config["d_model"],
        n_heads=config["n_heads"],
        d_ff=config["d_ff"],
        context_length=config["context_length"],
        rope_theta=config["rope_theta"],
        n_layers=config["n_layers"],
        vocab_size=config["vocab_size"],
        device=device
    ).to(device)

    # -----------------------------
    # 3. Optimizer & LR Scheduler
    # -----------------------------
    lr_scheduler = CosineSchedule(
        max_lr=config["max_learning_rate"],
        min_lr=config["min_learning_rate"],
        warmup_steps=config["lr_warmup_steps"],
        cycle_iters=config["cosine_cycle_iters"]
    )

    optimizer = AdamW(
        model.parameters(),
        lr=config["initial_lr"],
        betas=(config["adam_beta1"], config["adam_beta2"]),
        eps=config["eps"],
        weight_decay=config["weight_decay"]
    )

    loss_fn = CrossEntropyLoss()

    # -----------------------------
    # 4. Training loop
    # -----------------------------
    model.train()
    global_step = 0

    for epoch in range(config["epochs"]):

        if epoch >= 9:
            start_time = timeit.default_timer()

        for step in range(config["train_steps"]):

            # 更新学习率（基于 global_step）
            current_lr = lr_scheduler(global_step)
            for group in optimizer.param_groups:
                group["lr"] = current_lr

            # 取一个 batch
            x, y = train_loader.get_train_batch_data()
            x, y = x.to(device), y.to(device)

            # 前向传播
            logits = model(x)
            loss = loss_fn.forward(logits, y)

            # 反向传播
            optimizer.zero_grad()
            loss.backward()

            # 梯度裁剪（防止梯度爆炸）
            clip_gradient(model.parameters(), config["grad_clip"])
            optimizer.step()

            if device.type == "cuda":
                torch.cuda.synchronize()

            global_step += 1

            if step % 100 == 0:
                print(
                    f"[Epoch {epoch} | Step {step}] "
                    f"LR={current_lr:.6e}, Loss={loss.item():.4f}"
                )

        # epoch 计时
        if epoch >= 9:
            end_time = timeit.default_timer()
            with open(
                f"{config['d_model']}_{config['d_ff']}_"
                f"{config['n_layers']}_{config['n_heads']}_train_time.txt",
                "a"
            ) as f:
                f.write(f"Epoch {epoch} took {end_time - start_time:.2f}s\n")

        # WandB 记录
        wandb.log({"epoch": epoch, "train_loss": loss.item()})

        # -----------------------------
        # 5. Validation
        # -----------------------------
        if (epoch + 1) % config["val_interval"] == 0:
            model.eval()
            with torch.no_grad():
                for x, y in valid_loader.get_valid_batch_data_iter():
                    x, y = x.to(device), y.to(device)
                    logits = model(x)
                    val_loss = loss_fn.forward(logits, y)
                    wandb.log({"epoch": epoch, "val_loss": val_loss.item()})
            model.train()

        # -----------------------------
        # 6. Checkpoint
        # -----------------------------
        if (epoch + 1) % config["checkpoint_interval"] == 0:
            torch.save(
                {
                    "epoch": epoch,
                    "model_state_dict": model.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                },
                f"checkpoints/model_epoch_{epoch}_{TIMESTAMP}.pth"
            )
            print(f"Checkpoint saved at epoch {epoch}")

    torch.save(
        model.state_dict(),
        f"checkpoints/model_final_{TIMESTAMP}.pth"
    )
    print("Final checkpoint saved")


In [ ]:


# =====================================================
# CLI 入口
# =====================================================
if __name__ == "__main__":

    parser = argparse.ArgumentParser()
    parser.add_argument("--device", type=str, default="cuda:6")
    parser.add_argument("--epochs", type=int, default=40)
    parser.add_argument("--train_steps", type=int, default=2000)
    parser.add_argument("--batch_size", type=int, default=4)

    parser.add_argument("--d_model", type=int, default=768)
    parser.add_argument("--d_ff", type=int, default=3072)
    parser.add_argument("--n_layers", type=int, default=12)
    parser.add_argument("--n_heads", type=int, default=12)

    args = parser.parse_args()

    device = torch.device(
        args.device if torch.cuda.is_available() else "cpu"
    )

    config = init_wandb(
        d_model=args.d_model,
        d_ff=args.d_ff,
        n_layers=args.n_layers,
        n_heads=args.n_heads,
        batch_size=args.batch_size,
        epochs=args.epochs,
        train_steps=args.train_steps
    )

    train_model(config, device)
